In [1]:
from sklearn.preprocessing import RobustScaler, StandardScaler
from statsmodels.robust.scale import huber
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import re
import locale
import geopandas
plt.style.use('seaborn')
%matplotlib inline

pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)


# df_indicadores_pob = pd.read_parquet(f'../data/processed/df_indicadores_pob.parquet')
# df_final_pob = pd.read_parquet('../data/processed/df_final_pob.parquet')
# df_final_pob_melt = pd.read_parquet('../data/processed/df_final_pob_melt.parquet')
# df_final_pob_melt_PC = pd.read_parquet('../data/processed/df_final_pob_melt_PC.parquet')
# CCAA_CO = pd.read_parquet('../data/processed/CCAA_CO.parquet')
# MUNI_CO = pd.read_parquet('../data/processed/MUNI_CO.parquet')
# PROV_CO = pd.read_parquet('../data/processed/PROV_CO.parquet')



In [56]:
df_final_pob=pd.read_parquet('../data/main_processed/df_final_pob.parquet')

df_indicadores_pob=pd.read_parquet('../data/main_processed/df_indicadores_pob.parquet')

df_indicadores_pob_pivot=pd.read_parquet('../data/main_processed/df_indicadores_pob_pivot.parquet')

df_final_pob_melt=pd.read_parquet('../data/main_processed/df_final_pob_melt.parquet')

df_final_pob_melt_PC=pd.read_parquet('../data/main_processed/df_final_pob_melt_PC.parquet')

CCAA_CO = pd.read_parquet('../data/main_processed/CCAA_CO.parquet')
MUNI_CO = pd.read_parquet('../data/main_processed/MUNI_CO.parquet')
PROV_CO = pd.read_parquet('../data/main_processed/PROV_CO.parquet')

df_final_pob_dropdown=pd.read_parquet('../data/main_processed/df_final_pob_dropdown.parquet')

df_final_pob_dropdown_c=pd.read_parquet('../data/main_processed/df_final_pob_dropdown_c.parquet')

df_final_pob_poblaciontext=pd.read_parquet('../data/main_processed/df_final_pob_poblaciontext.parquet')




In [8]:
CCAA_types='Andalucía'

In [78]:
%%timeit

pob_c=df_final_pob.pivot_table(index=['CCAA'] ,values=['Población 2018'] ,aggfunc=sum).reset_index()
df_count_c_pc=df_final_pob_melt.pivot_table(index=['CCAA','Descripción'] ,values=['coste_efectivo'] ,aggfunc=sum).reset_index()
df_count_c_pc = pd.merge(df_count_c_pc, pob_c, on='CCAA', how='left')
df_count_c_pc['PC_TOTAL'] = df_count_c_pc['coste_efectivo']/df_count_c_pc['Población 2018']

df_count_c_pc

28.4 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [80]:
pob_c=df_final_pob.pivot_table(index=['CCAA'] ,values=['Población 2018'] ,aggfunc=sum).reset_index()
df_count_c_pc=df_final_pob_melt.pivot_table(index=['CCAA','Descripción'] ,values=['coste_efectivo'] ,aggfunc=sum).reset_index()
df_count_c_pc = pd.merge(df_count_c_pc, pob_c, on='CCAA', how='left')
df_count_c_pc['PC_TOTAL'] = df_count_c_pc['coste_efectivo']/df_count_c_pc['Población 2018']

df_count_c_pc

,CCAA,Descripción,coste_efectivo,Población 2018,PC_TOTAL
0,Andalucía,Abastecimiento de agua potable,3.649560e+08,7677889.0,47.533372
1,Andalucía,"Abastos, mercados, lonjas",2.246864e+07,7677889.0,2.926408
2,Andalucía,Acceso a los núcleos de población,4.002181e+07,7677889.0,5.212606
3,Andalucía,Actividades funerarias,2.334983e+07,7677889.0,3.041179
4,Andalucía,Alcantarillado,1.164123e+08,7677889.0,15.162015
5,Andalucía,Alumbrado público,2.319521e+08,7677889.0,30.210406
6,Andalucía,Biblioteca pública,4.594739e+07,7677889.0,5.984378
7,Andalucía,Cementerio,4.236281e+07,7677889.0,5.517508
8,Andalucía,Comercio ambulante,2.726953e+06,7677889.0,0.355170
...,...,...,...,...,...


In [79]:
df_count_c_pc

# 149 ms ± 17.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)



# 10.8 ms ± 256 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

NameError: name 'df_count_c_pc' is not defined

In [9]:
df_count_c=df_final_pob.pivot_table(index=['CCAA'] ,values=['TOTAL','Población 2018'] ,aggfunc=sum).reset_index()
df_count_c['PC_TOTAL'] = df_count_c.apply(lambda new: round(new['TOTAL']/new['Población 2018'],), axis=1)
df_count_c=df_count_c.sort_values(by='PC_TOTAL',ascending=False)
df_count_c['CCAA'] = df_count_c['CCAA'].astype('object')


df_count_c_new_n=df_count_c.loc[(df_count_c['PC_TOTAL']==df_count_c['PC_TOTAL'].quantile(0.75, interpolation='nearest'))|\
                (df_count_c['PC_TOTAL']==df_count_c['PC_TOTAL'].quantile(0.50, interpolation='nearest'))|\
                 (df_count_c['PC_TOTAL']==df_count_c['PC_TOTAL'].quantile(0.25, interpolation='nearest'))]


df = df_count_c_new_n
df = df.append(df_count_c.loc[df_count_c['CCAA']==CCAA_types])

In [10]:
df

,CCAA,Población 2018,TOTAL,PC_TOTAL
6,Cataluña,7271555.0,5.646433e+09,777
13,Principado de Asturias,987025.0,6.729865e+08,682
4,Castilla y León,1947252.0,1.190997e+09,612
0,Andalucía,7677889.0,5.554223e+09,723


In [22]:
df_count_p=df_final_pob.pivot_table(index=['Provincia'] ,values=['TOTAL','Población 2018'] ,aggfunc=sum).reset_index()
df_count_p['PC_TOTAL'] = df_count_p.apply(lambda new: round(new['TOTAL']/new['Población 2018'],), axis=1)
df_count_p['Provincia'] = df_count_p['Provincia'].astype('object')

df_count_p_new_n=df_count_p.loc[(df_count_p['PC_TOTAL']==df_count_p['PC_TOTAL'].quantile(0.75, interpolation='nearest'))|\
                (df_count_p['PC_TOTAL']==df_count_p['PC_TOTAL'].quantile(0.51, interpolation='nearest'))|\
                 (df_count_p['PC_TOTAL']==df_count_p['PC_TOTAL'].quantile(0.25, interpolation='nearest'))]


In [23]:
df_count_p_new_n

,Provincia,Población 2018,TOTAL,PC_TOTAL
22,Jaén,549573.0,3.239394e+08,589
24,Lleida,377734.0,2.820167e+08,747
36,Segovia,118354.0,7.733025e+07,653


In [24]:
cohorte='más de 500.000'
municipio_types = 'Madrid'



In [44]:
cohorte = df_final_pob.loc[df_final_pob['Nombre Ente Principal'] == municipio_types , 'cohorte_pob'].unique().to_list()[0]

df = df_final_pob[['Nombre Ente Principal' , 'cohorte_pob' , 'PC_TOTAL']].loc[
    (df_final_pob['cohorte_pob'] == cohorte) & (df_final_pob['PC_TOTAL'] > 0)].sort_values(by='PC_TOTAL' ,
                                                                                           ascending=False)
df['Nombre Ente Principal'] = df['Nombre Ente Principal'].astype('object')
# df['PC_TOTAL'] = df.apply(lambda new: round(new['PC_TOTAL'] , ) , axis=1)
df2=df.loc[(df['PC_TOTAL']==df['PC_TOTAL'].quantile(0.75, interpolation='nearest'))|\
                (df['PC_TOTAL']==df['PC_TOTAL'].quantile(0.50, interpolation='nearest'))|\
                 (df['PC_TOTAL']==df['PC_TOTAL'].quantile(0.25, interpolation='nearest'))]




df2 = df2.append(df.loc[df['Nombre Ente Principal']==municipio_types])
df=df2
df.iloc[3 , 0] = f'{municipio_types}.'




In [45]:
df

,Nombre Ente Principal,cohorte_pob,PC_TOTAL
645,Sevilla,más de 500.000,994.624372
2825,Barcelona,más de 500.000,872.550323
1115,Zaragoza,más de 500.000,863.149121
4186,Madrid.,más de 500.000,1118.909850


In [47]:
partida_de_coste_types='Abastecimiento de agua potable'

In [50]:
pob_c=df_final_pob.pivot_table(index=['CCAA'] ,values=['Población 2018'] ,aggfunc=sum).reset_index()
df_count_c_pc=df_final_pob_melt.pivot_table(index=['CCAA','Descripción'] ,values=['coste_efectivo'] ,aggfunc=sum).reset_index()
df_count_c_pc = pd.merge(df_count_c_pc, pob_c, on='CCAA', how='left')
df_count_c_pc['PC_TOTAL'] = df_count_c_pc.apply(lambda new: new['coste_efectivo']/new['Población 2018'], axis=1)
df_count_c_pc['CCAA'] = df_count_c_pc['CCAA'].astype('object')


df = df_count_c_pc

df = df.loc[df['Descripción'] == partida_de_coste_types].sort_values(by='PC_TOTAL' ,ascending=False)


df =df.loc[(df['PC_TOTAL']==df['PC_TOTAL'].quantile(0.90, interpolation='nearest'))|\
                (df['PC_TOTAL']==df['PC_TOTAL'].quantile(0.75, interpolation='nearest'))|\
                 (df['PC_TOTAL']==df['PC_TOTAL'].quantile(0.25, interpolation='nearest'))|\
                 (df['PC_TOTAL']==df['PC_TOTAL'].quantile(0.50, interpolation='nearest'))]
df['PC_TOTAL'] = df.apply(lambda new: round(new['PC_TOTAL'],), axis=1)




df 





,CCAA,Descripción,coste_efectivo,Población 2018,PC_TOTAL
86,Canarias,Abastecimiento de agua potable,1.523355e+08,1932728.0,79
430,Galicia,Abastecimiento de agua potable,1.156006e+08,2335771.0,49
43,Aragón,Abastecimiento de agua potable,5.280455e+07,1235915.0,43
559,Principado de Asturias,Abastecimiento de agua potable,4.024742e+07,987025.0,41


In [52]:
PROV_types='Murcia'

In [56]:
pob_p=df_final_pob.pivot_table(index=['Provincia'] ,values=['Población 2018'] ,aggfunc=sum).reset_index()
df_count_p_pc=df_final_pob_melt.pivot_table(index=['Provincia','Descripción'] ,values=['coste_efectivo'] ,aggfunc=sum).reset_index()
df_count_p_pc = pd.merge(df_count_p_pc, pob_p, on='Provincia', how='left')
df_count_p_pc['PC_TOTAL'] = df_count_p_pc.apply(lambda new: new['coste_efectivo']/new['Población 2018'], axis=1)
df_count_p_pc['Provincia'] = df_count_p_pc['Provincia'].astype('object')

df = df_count_p_pc

df = df.loc[df['Descripción'] == partida_de_coste_types].sort_values(by='PC_TOTAL' , ascending=False)

df2 = df.loc[(df['PC_TOTAL'] == df['PC_TOTAL'].quantile(0.75 , interpolation='nearest')) | \
             (df['PC_TOTAL'] == df['PC_TOTAL'].quantile(0.50 , interpolation='nearest')) | \
             (df['PC_TOTAL'] == df['PC_TOTAL'].quantile(0.25 , interpolation='nearest'))]
df2 = df2.sort_values(by='PC_TOTAL' , ascending=False)

df2 = df2.append(df.loc[df['Provincia'] == PROV_types])
# df2['PC_TOTAL'] = df2.apply(lambda new: round(new['PC_TOTAL'] , ) , axis=1)
df2.iloc[3 , 0] = f'{PROV_types}.'



df2

,Provincia,Descripción,coste_efectivo,Población 2018,PC_TOTAL
428,Ciudad Real,Abastecimiento de agua potable,2.166432e+07,422487.0,51.278081
129,Asturias,Abastecimiento de agua potable,4.024742e+07,987025.0,40.776494
1582,Sevilla,Abastecimiento de agua potable,6.333921e+07,1775103.0,35.681994
1156,Murcia.,Abastecimiento de agua potable,1.517914e+08,1423404.0,106.639748


In [11]:
cohorte='0-1000' 
municipio_types = 'Abadía'
partida_de_coste_types='Acceso a los núcleos de población'


In [84]:
cohorte = df_final_pob_melt_PC.loc[df_final_pob_melt_PC['Nombre Ente Principal'] == municipio_types , 'cohorte_pob'].unique().to_list()[0]

df = df_final_pob_melt_PC[['Nombre Ente Principal' , 'cohorte_pob' , 'Descripción' , 'coste_efectivo_PC']].loc[
    (df_final_pob_melt_PC['cohorte_pob'] == cohorte)  & ( df_final_pob_melt_PC['Descripción'] == partida_de_coste_types)].sort_values(by='coste_efectivo_PC' , ascending=False)
df['Nombre Ente Principal'] = df['Nombre Ente Principal'].astype('object')

df2 = df.loc[(df['coste_efectivo_PC'] == df['coste_efectivo_PC'].quantile(0.75 , interpolation='nearest')) | \
             (df['coste_efectivo_PC'] == df['coste_efectivo_PC'].quantile(0.50 , interpolation='nearest')) | \
             (df['coste_efectivo_PC'] == df['coste_efectivo_PC'].quantile(0.25 , interpolation='nearest'))]

df2 = df2.sort_values(by='coste_efectivo_PC' , ascending=False)
df2.drop_duplicates(subset='coste_efectivo_PC' , keep='last',inplace=True)

df = df2.append(df.loc[df['Nombre Ente Principal'] == municipio_types])

df.iloc[3 , 0] = f'{municipio_types}.'
df['coste_efectivo_PC'] = df.apply(lambda new: round(new['coste_efectivo_PC'] , ) , axis=1)



In [85]:
df

,Nombre Ente Principal,cohorte_pob,Descripción,coste_efectivo_PC
10672,Bardallur,0-1.000,Acceso a los núcleos de población,29
11768,Alba de Yeltes,0-1.000,Acceso a los núcleos de población,6
13026,Estaràs,0-1.000,Acceso a los núcleos de población,1
13448,Abadía.,0-1.000,Acceso a los núcleos de población,43


In [87]:
df_zoom_pob = geopandas.read_file('../data/main_raw/shapefiles_espana_municipios.geojson')

df_muni_co_zoom=df_final_pob[['CCAA','Provincia','Nombre Ente Principal','codigo_geo','Población 2018']]
df_muni_co_zoom[df_muni_co_zoom.select_dtypes(['category']).columns] = \
            df_muni_co_zoom.select_dtypes(['category']).apply(lambda x: x.astype('object'))

df_muni_co_zoom['codigo_geo'] = df_muni_co_zoom['codigo_geo'].astype('int64')
df_zoom_pob['codigo_geo']= df_zoom_pob['f_codmun']
df_zoom_pob = pd.merge(df_zoom_pob , df_muni_co_zoom, on='codigo_geo', how='left')

/home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [97]:
df = df_final_pob_melt_PC#[df_final_pob_melt_PC['Población 2018']>5000]
df['Población'] = df['Población 2018']

df= df.loc[(df['Descripción']==partida_de_coste_types)& (df['coste_efectivo_PC'] > 0)]
df['coste_efectivo_PC'] = df.apply(lambda new: round(new['coste_efectivo_PC'] , ) , axis=1)

df

/home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,CCAA,Provincia,Tipo Ente Principal,Nombre Ente Principal,Código Ente Principal,cohorte_pob,codigo_geo,Descripción,coste_efectivo_PC,Población 2018,Población
9738,Andalucía,Almería,Ayuntamiento,Abla,01-04-001-AA-000,1.000-2.000,4001,Acceso a los núcleos de población,13,1235.0,1235.0
9739,Andalucía,Almería,Ayuntamiento,Abrucena,01-04-002-AA-000,1.000-2.000,4002,Acceso a los núcleos de población,5,1150.0,1150.0
9740,Andalucía,Almería,Ayuntamiento,Adra,01-04-003-AA-000,20.000-50.000,4003,Acceso a los núcleos de población,24,24859.0,24859.0
9741,Andalucía,Almería,Ayuntamiento,Albánchez,01-04-004-AA-000,0-1.000,4004,Acceso a los núcleos de población,54,753.0,753.0
9742,Andalucía,Almería,Ayuntamiento,Alboloduy,01-04-005-AA-000,0-1.000,4005,Acceso a los núcleos de población,6,621.0,621.0
9743,Andalucía,Almería,Ayuntamiento,Albox,01-04-006-AA-000,10.000-20.000,4006,Acceso a los núcleos de población,7,11696.0,11696.0
9744,Andalucía,Almería,Ayuntamiento,Alcolea,01-04-007-AA-000,0-1.000,4007,Acceso a los núcleos de población,35,814.0,814.0
9745,Andalucía,Almería,Ayuntamiento,Alcóntar,01-04-008-AA-000,0-1.000,4008,Acceso a los núcleos de población,34,528.0,528.0
9746,Andalucía,Almería,Ayuntamiento,Alcudia de Monteagud,01-04-009-AA-000,0-1.000,4009,Acceso a los núcleos de población,62,135.0,135.0
...,...,...,...,...,...,...,...,...,...,...,...


In [3]:
df_final_pob_melt_PC.columns

Index(['CCAA', 'Provincia', 'Tipo Ente Principal', 'Nombre Ente Principal',
       'Código Ente Principal', 'cohorte_pob', 'codigo_geo', 'Descripción',
       'coste_efectivo_PC', 'Población 2018'],
      dtype='object')

In [7]:
%%timeit

df = df_final_pob_melt_PC
df['coste_efectivo_PC'] = df.apply(lambda new: round(new['coste_efectivo_PC'] , ) , axis=1)




12.1 s ± 421 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit

df = df_final_pob_melt_PC
df['coste_efectivo_PC'] = round(df['coste_efectivo_PC'],)


369 µs ± 5.34 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [17]:
df_final_pob_round_melt_PC = df_final_pob_melt_PC[['codigo_geo','CCAA', 'Provincia', 'Nombre Ente Principal','Población 2018','coste_efectivo_PC','Descripción']]
df_final_pob_round_melt_PC['Población'] = df_final_pob_round_melt_PC['Población 2018']
df_final_pob_round_melt_PC['coste_efectivo_PC'] = round(df_final_pob_round_melt_PC['coste_efectivo_PC'] , )


df = df_final_pob_round_melt_PC#[df_final_pob_round_melt_PC['Población']>5000]
            # df['Población'] = df['Población 2018']
df= df.loc[(df['Descripción']==partida_de_coste_types)& (df['coste_efectivo_PC'] > 0)]

df

/home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,codigo_geo,CCAA,Provincia,Nombre Ente Principal,Población 2018,coste_efectivo_PC,Descripción,Población
9738,4001,Andalucía,Almería,Abla,1235.0,13,Acceso a los núcleos de población,1235.0
9739,4002,Andalucía,Almería,Abrucena,1150.0,5,Acceso a los núcleos de población,1150.0
9740,4003,Andalucía,Almería,Adra,24859.0,24,Acceso a los núcleos de población,24859.0
9741,4004,Andalucía,Almería,Albánchez,753.0,54,Acceso a los núcleos de población,753.0
9742,4005,Andalucía,Almería,Alboloduy,621.0,6,Acceso a los núcleos de población,621.0
9743,4006,Andalucía,Almería,Albox,11696.0,7,Acceso a los núcleos de población,11696.0
9744,4007,Andalucía,Almería,Alcolea,814.0,35,Acceso a los núcleos de población,814.0
9745,4008,Andalucía,Almería,Alcóntar,528.0,34,Acceso a los núcleos de población,528.0
9746,4009,Andalucía,Almería,Alcudia de Monteagud,135.0,62,Acceso a los núcleos de población,135.0
...,...,...,...,...,...,...,...,...


In [48]:
df_zoom_pob = geopandas.read_file('../data/main_raw/shapefiles_espana_municipios.geojson')

df_muni_co_zoom=df_final_pob[['CCAA','Provincia','Nombre Ente Principal','codigo_geo','Población 2018']]
df_muni_co_zoom[df_muni_co_zoom.select_dtypes(['category']).columns] = \
            df_muni_co_zoom.select_dtypes(['category']).apply(lambda x: x.astype('object'))

df_muni_co_zoom['codigo_geo'] = df_muni_co_zoom['codigo_geo'].astype('int64')
df_zoom_pob['codigo_geo']= df_zoom_pob['f_codmun']
df_zoom_pob = pd.merge(df_zoom_pob , df_muni_co_zoom, on='codigo_geo', how='left')

df_zoom_pob=df_zoom_pob.loc[df_zoom_pob['CCAA'].isnull()==False]
df_zoom_pob=df_zoom_pob[['f_codmun', 'geometry','codigo_geo', 'CCAA', 'Provincia', 'Nombre Ente Principal', 'Población 2018']]


df['codigo_geo']=df['codigo_geo'].astype('int64')

df_zoom_pob= pd.merge(df_zoom_pob , df, on='codigo_geo', how='left',suffixes=('','_y'))
df_zoom_pob=df_zoom_pob.loc[df_zoom_pob['CCAA_y'].isnull()==False]

df_zoom_pob.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2128 entries, 6 to 4864
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   f_codmun                 2128 non-null   int64   
 1   geometry                 2128 non-null   geometry
 2   codigo_geo               2128 non-null   int64   
 3   CCAA                     2128 non-null   object  
 4   Provincia                2128 non-null   object  
 5   Nombre Ente Principal    2128 non-null   object  
 6   Población 2018           2128 non-null   float64 
 7   CCAA_y                   2128 non-null   category
 8   Provincia_y              2128 non-null   category
 9   Nombre Ente Principal_y  2128 non-null   category
 10  Población 2018_y         2128 non-null   float64 
 11  coste_efectivo_PC        2128 non-null   float64 
 12  Descripción              2128 non-null   category
 13  Población                2128 non-null   float64 
dtype

/home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [35]:
df_zoom_pob=df_zoom_pob.loc[df_zoom_pob['Población'].isnull()==False]

df_zoom_pob

,cartodb_id,nameunit,natcode,f_codmun,codmun,codccaa,codmun_no_0,codccaa_no_0,f_codmun_no_0,geometry,codigo_geo,CCAA_x,Provincia_x,Nombre Ente Principal_x,Población 2018_x,CCAA_y,Provincia_y,Nombre Ente Principal_y,Población 2018_y,coste_efectivo_PC,Descripción,Población
9,664,Trigueros,34012121070,21070,070,21,70,21,2170,"MULTIPOLYGON (((-6.97310 37.47089, -6.97061 37...",21070,Andalucía,Huelva,Trigueros,7782.0,Andalucía,Huelva,Trigueros,7782.0,9.0,Acceso a los núcleos de población,7782.0
12,1766,Monfero,34121515050,15050,050,15,50,15,1550,"MULTIPOLYGON (((-8.10302 43.39414, -8.11028 43...",15050,Galicia,"Coruña, A",Monfero,1962.0,Galicia,"Coruña, A",Monfero,1962.0,7.0,Acceso a los núcleos de población,1962.0
13,1767,Mugardos,34121515051,15051,051,15,51,15,1551,"MULTIPOLYGON (((-8.27908 43.45892, -8.26795 43...",15051,Galicia,"Coruña, A",Mugardos,5307.0,Galicia,"Coruña, A",Mugardos,5307.0,15.0,Acceso a los núcleos de población,5307.0
16,6815,Casinos,34104646089,46089,089,46,89,46,4689,"MULTIPOLYGON (((-0.79331 39.72037, -0.73605 39...",46089,Comunitat Valenciana,Valencia / València,Casinos,2773.0,Comunitat Valenciana,Valencia / València,Casinos,2773.0,16.0,Acceso a los núcleos de población,2773.0
17,1091,Ribatejada,34132828122,28122,122,28,122,28,28122,"MULTIPOLYGON (((-3.48268 40.67041, -3.46825 40...",28122,Comunidad de Madrid,Madrid,Ribatejada,706.0,Comunidad de Madrid,Madrid,Ribatejada,706.0,29.0,Acceso a los núcleos de población,706.0
19,480,La Floresta,34092525092,25092,092,25,92,25,2592,"MULTIPOLYGON (((0.91396 41.51418, 0.93414 41.5...",25092,Cataluña,Lleida,Floresta (La),154.0,Cataluña,Lleida,Floresta (La),154.0,3.0,Acceso a los núcleos de población,154.0
21,1414,Poio,34123636041,36041,041,36,41,36,3641,"MULTIPOLYGON (((-8.75822 42.41990, -8.74939 42...",36041,Galicia,Pontevedra,Poio,17018.0,Galicia,Pontevedra,Poio,17018.0,2.0,Acceso a los núcleos de población,17018.0
23,127,Roquetas de Mar,34010404079,4079,079,04,79,4,479,"MULTIPOLYGON (((-2.66451 36.72317, -2.66129 36...",4079,Andalucía,Almería,Roquetas de Mar,94925.0,Andalucía,Almería,Roquetas de Mar,94925.0,5.0,Acceso a los núcleos de población,94925.0
25,545,Lozoya,34132828076,28076,076,28,76,28,2876,"MULTIPOLYGON (((-3.84451 40.98235, -3.82909 40...",28076,Comunidad de Madrid,Madrid,Lozoya,559.0,Comunidad de Madrid,Madrid,Lozoya,559.0,4.0,Acceso a los núcleos de población,559.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
